In [1]:
import os, time, sys
from vllm import LLM
sys.path.append('../elyra_pipeline/')
from postgres_utilities import connect_db, close_db, execute_sql_results, execute_sql_results_np

INFO 05-05 13:36:15 [__init__.py:239] Automatically detected platform cuda.


In [2]:
model_name = "GritLM/GritLM-7B"
number_data_rows = 1000

In [3]:
llm = LLM(model=model_name, task="embed", max_model_len=4096, quantization="bitsandbytes", load_format="bitsandbytes")

WARNING 05-05 13:36:27 [config.py:830] bitsandbytes quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 05-05 13:36:27 [arg_utils.py:1658] --task embed is not supported by the V1 Engine. Falling back to V0. 
INFO 05-05 13:36:28 [llm_engine.py:240] Initializing a V0 LLM engine (v0.8.5.post1) with config: model='GritLM/GritLM-7B', speculative_config=None, tokenizer='GritLM/GritLM-7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.BITSANDBYTES, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=bitsandbytes, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False,

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 05-05 13:36:34 [model_runner.py:1140] Model loading took 3.6059 GiB and 5.118952 seconds


In [4]:
try:
    conn = connect_db()
    sql = f"select productdisplayname from products limit {number_data_rows}"
    results = execute_sql_results_np(conn, sql)
except Exception as e:
    raise e
finally:
    close_db(conn)

In [5]:
test_description = "Starting batch embedding test using local vllm"
print(test_description)
print(len(test_description) * "-") 
tic = time.perf_counter()
print("Embeddings computed ...", end=" ")
descriptions = list()
for index, product in enumerate(results):
    descriptions.append(product[0])

batch_size = 1000
start_offset = 0
end_offset = batch_size
embeddings = list()
while start_offset < number_data_rows:
    print(start_offset, end_offset, end=" ")
    embeddings = embeddings + llm.embed(descriptions[start_offset:end_offset])
    start_offset = end_offset
    end_offset = end_offset + batch_size
    if end_offset > number_data_rows:
        end_offset = number_data_rows

assert len(embeddings) == index+1
for e in embeddings:
    assert(e.outputs.hidden_size == 4096)

toc = time.perf_counter()
print("")
print(f"Time to compute {number_data_rows} embeddings: {toc - tic:0.4f} seconds")
print(len(test_description) * "-") 

Starting batch embedding test using local vllm
----------------------------------------------
Embeddings computed ... 0 1000 

Processed prompts:   0%|          | 0/1000 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s…


Time to compute 1000 embeddings: 3.6617 seconds
----------------------------------------------
